# MIKE IO 1D Pandas DataFrame examples

Results in MIKE IO 1D are fundamentally extracted as Pandas DataFrames. This notebook shows
specific methods for working with those DataFrames.

In [1]:
from mikeio1d import Res1D

res = Res1D('../tests/testdata/network_river.res1d')

### Column modes
MIKE IO 1D supports different types of column headers. Depending on your use case, one or the other may be preferable. There are four main modes:
- string (default)
- all
- compact
- timeseries

These are illustrated in the following cells.

In [2]:
# The default column mode concatenates information about the column with a ':' delimiter.   
df = res.reaches.Discharge.read()
df.head(2)

,Discharge:basin_left1:1.002,Discharge:basin_left1:17.889,Discharge:basin_left1:57.3595,Discharge:basin_left1:101.493,Discharge:basin_left1:144.075,Discharge:basin_left1:175.496,Discharge:basin_left1:197.549,Discharge:basin_left1:228.411,Discharge:basin_left1:255.612,Discharge:basin_left1:274.626,...,Discharge:tributary:95,Discharge:tributary:110,Discharge:tributary:125,Discharge:tributary:140,Discharge:tributary:175,Discharge:tributary:225,Discharge:tributary:275,Discharge:tributary:350,Discharge:tributary:425,Discharge:tributary:475
2000-02-18 00:06:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2000-02-18 00:16:00,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,...,0.013857,-0.0,0.050522,0.053607,0.090468,0.107519,0.144026,0.177496,0.191049,0.197755


In [3]:
# A hierarchical column mode is also supported. Only relevant levels are included with 'compact'. For full hierarchy use 'all'.
df = res.reaches.Discharge.read(column_mode='compact')
df.head(2)

quantity              Discharge                                          \
group                     Reach                                           
name                basin_left1                                           
chainage               1.0020    17.8890   57.3595   101.4935  144.0745   
tag                   0.0-246.6 0.0-246.6 0.0-246.6 0.0-246.6 0.0-246.6   
2000-02-18 00:06:00         0.0       0.0       0.0       0.0       0.0   
2000-02-18 00:16:00        -0.0      -0.0      -0.0      -0.0      -0.0   

quantity                                                                   \
group                                                                       
name                                                                        
chainage             175.4965  197.5490  228.4110    255.6120    274.6260   
tag                 0.0-246.6 0.0-246.6 0.0-246.6 246.6-413.6 246.6-413.6   
2000-02-18 00:06:00       0.0       0.0       0.0         0.0         0.0   
2000-02-18 00:16:00      -0.0      -0.0      -0.0        -0.0        -0.0   

quantity             ...                                              \
group                ...                                               
name                 ...  tributary                                    
chainage             ...   95.0000    110.0000   125.0000   140.0000   
tag                  ... 50.0-500.0 50.0-500.0 50.0-500.0 50.0-500.0   
2000-02-18 00:06:00  ...   0.000000        0.0   0.000000   0.000000   
2000-02-18 00:16:00  ...   0.013857       -0.0   0.050522   0.053607   

quantity                                                                    \
group                                                                        
name                                                                         
chainage              175.0000   225.0000   275.0000   350.0000   425.0000   
tag                 50.0-500.0 50.0-500.0 50.0-500.0 50.0-500.0 50.0-500.0   
2000-02-18 00:06:00   0.000000   0.000000   0.000000   0.000000   0.000000   
2000-02-18 00:16:00   0.090468   0.107519   0.144026   0.177496   0.191049   

quantity                        
group                           
name                            
chainage              475.0000  
tag                 50.0-500.0  
2000-02-18 00:06:00   0.000000  
2000-02-18 00:16:00   0.197755  

[2 rows x 101 columns]

In [4]:
# It's also possible to have a TimeSeriesId column index.
# TimeSeriesId is an object that uniquely identifies each time series.
df = res.reaches.Discharge.read(column_mode='timeseries')
df.head(2)

,"TimeSeriesId(quantity='Discharge', group='Reach', name='basin_left1', chainage=1.002, tag='0.0-246.6', duplicate=0, derived=False)","TimeSeriesId(quantity='Discharge', group='Reach', name='basin_left1', chainage=17.889, tag='0.0-246.6', duplicate=0, derived=False)","TimeSeriesId(quantity='Discharge', group='Reach', name='basin_left1', chainage=57.3595, tag='0.0-246.6', duplicate=0, derived=False)","TimeSeriesId(quantity='Discharge', group='Reach', name='basin_left1', chainage=101.4935, tag='0.0-246.6', duplicate=0, derived=False)","TimeSeriesId(quantity='Discharge', group='Reach', name='basin_left1', chainage=144.0745, tag='0.0-246.6', duplicate=0, derived=False)","TimeSeriesId(quantity='Discharge', group='Reach', name='basin_left1', chainage=175.4965, tag='0.0-246.6', duplicate=0, derived=False)","TimeSeriesId(quantity='Discharge', group='Reach', name='basin_left1', chainage=197.54899999999998, tag='0.0-246.6', duplicate=0, derived=False)","TimeSeriesId(quantity='Discharge', group='Reach', name='basin_left1', chainage=228.411, tag='0.0-246.6', duplicate=0, derived=False)","TimeSeriesId(quantity='Discharge', group='Reach', name='basin_left1', chainage=255.612, tag='246.6-413.6', duplicate=0, derived=False)","TimeSeriesId(quantity='Discharge', group='Reach', name='basin_left1', chainage=274.626, tag='246.6-413.6', duplicate=0, derived=False)",...,"TimeSeriesId(quantity='Discharge', group='Reach', name='tributary', chainage=95.0, tag='50.0-500.0', duplicate=0, derived=False)","TimeSeriesId(quantity='Discharge', group='Reach', name='tributary', chainage=110.0, tag='50.0-500.0', duplicate=0, derived=False)","TimeSeriesId(quantity='Discharge', group='Reach', name='tributary', chainage=125.0, tag='50.0-500.0', duplicate=0, derived=False)","TimeSeriesId(quantity='Discharge', group='Reach', name='tributary', chainage=140.0, tag='50.0-500.0', duplicate=0, derived=False)","TimeSeriesId(quantity='Discharge', group='Reach', name='tributary', chainage=175.0, tag='50.0-500.0', duplicate=0, derived=False)","TimeSeriesId(quantity='Discharge', group='Reach', name='tributary', chainage=225.0, tag='50.0-500.0', duplicate=0, derived=False)","TimeSeriesId(quantity='Discharge', group='Reach', name='tributary', chainage=275.0, tag='50.0-500.0', duplicate=0, derived=False)","TimeSeriesId(quantity='Discharge', group='Reach', name='tributary', chainage=350.0, tag='50.0-500.0', duplicate=0, derived=False)","TimeSeriesId(quantity='Discharge', group='Reach', name='tributary', chainage=425.0, tag='50.0-500.0', duplicate=0, derived=False)","TimeSeriesId(quantity='Discharge', group='Reach', name='tributary', chainage=475.0, tag='50.0-500.0', duplicate=0, derived=False)"
2000-02-18 00:06:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2000-02-18 00:16:00,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,...,0.013857,-0.0,0.050522,0.053607,0.090468,0.107519,0.144026,0.177496,0.191049,0.197755


### MIKE IO 1D extends Pandas by providing a '.m1d' accessor on all DataFrames.
The accessor provides several useful methods for working with DataFrames within MIKE IO 1D:
- .m1d.query()
- .m1d.compact()
- .m1d.groupby()
- .m1d.groupby_chainage()
- .m1d.agg_chainage()

These methods are illustrated below.

In [5]:
# To use the .m1d accessor, the DataFrame must have a MultiIndex column (e.g. column_mode='all' or 'compact').
# The .m1d accessor exists on the DataFrame itself.
df = res.read(column_mode='all')
df.m1d

#### .m1d.compact()

In [6]:
# Let's make some queries on the DataFrame itself with help from the .m1d accessor.
# Let's read the entire file into a DataFrame with column_mode='all'.
df = res.read(column_mode='all')
df.head(2)

quantity                  WaterLevel                                    \
group                           Node                                     
name                'basin_left1', 0 'basin_left1', 483.74285236705026   
chainage                         NaN                               NaN   
tag                                                                      
duplicate                          0                                 0   
derived                        False                             False   
2000-02-18 00:06:00        57.426998                         53.495998   
2000-02-18 00:16:00        57.426998                         53.495998   

quantity                                                                \
group                                                                    
name                'basin_left2', 0 'basin_left2', 441.71489580142429   
chainage                         NaN                               NaN   
tag                                                                      
duplicate                          0                                 0   
derived                        False                             False   
2000-02-18 00:06:00        54.002998                         59.200001   
2000-02-18 00:16:00        54.002998                         59.200001   

quantity                                                                  \
group                                                                      
name                'basin_right', -10 'basin_right', 720 'river', 53100   
chainage                           NaN                NaN            NaN   
tag                                                                        
duplicate                            0                  0              0   
derived                          False              False          False   
2000-02-18 00:06:00             53.652          58.342999      51.835999   
2000-02-18 00:16:00             53.652          58.342999      51.836006   

quantity                                                        \
group                                                            
name                'river', 55124.2276598819 'tributary', 500   
chainage                                  NaN              NaN   
tag                                                              
duplicate                                   0                0   
derived                                 False            False   
2000-02-18 00:06:00                    53.194        55.480000   
2000-02-18 00:16:00                    53.194        55.946224   

quantity                                   ... FlowVelocity              \
group                                      ...        Reach               
name                'basin_left1', 246.61  ...    tributary               
chainage                              NaN  ...  175.00000   225.00000     
tag                                        ...   50.0-500.0  50.0-500.0   
duplicate                               0  ...            0           0   
derived                             False  ...        False       False   
2000-02-18 00:06:00             53.065304  ...     0.000000    0.000000   
2000-02-18 00:16:00             53.065304  ...     0.608204    0.517946   

quantity                                                             \
group                                                                 
name                                                                  
chainage            275.00000   350.00000   425.00000   475.00000     
tag                  50.0-500.0  50.0-500.0  50.0-500.0  50.0-500.0   
duplicate                     0           0           0           0   
derived                   False       False       False       False   
2000-02-18 00:06:00    0.000000    0.000000    0.000000    0.000000   
2000-02-18 00:16:00    0.449898    0.446345    0.419857    0.417569   

quantity            Variable:TwoTimeSensorGateLevel  \
group          

In [7]:
# That's a lot of detail included and is a bit busy. We can use .m1d.compact() to remove redundant levels.
df = df.m1d.compact()
df.head(2)

quantity                  WaterLevel                                    \
group                           Node                                     
name                'basin_left1', 0 'basin_left1', 483.74285236705026   
chainage                         NaN                               NaN   
tag                                                                      
2000-02-18 00:06:00        57.426998                         53.495998   
2000-02-18 00:16:00        57.426998                         53.495998   

quantity                                                                \
group                                                                    
name                'basin_left2', 0 'basin_left2', 441.71489580142429   
chainage                         NaN                               NaN   
tag                                                                      
2000-02-18 00:06:00        54.002998                         59.200001   
2000-02-18 00:16:00        54.002998                         59.200001   

quantity                                                                  \
group                                                                      
name                'basin_right', -10 'basin_right', 720 'river', 53100   
chainage                           NaN                NaN            NaN   
tag                                                                        
2000-02-18 00:06:00             53.652          58.342999      51.835999   
2000-02-18 00:16:00             53.652          58.342999      51.836006   

quantity                                                        \
group                                                            
name                'river', 55124.2276598819 'tributary', 500   
chainage                                  NaN              NaN   
tag                                                              
2000-02-18 00:06:00                    53.194        55.480000   
2000-02-18 00:16:00                    53.194        55.946224   

quantity                                   ... FlowVelocity              \
group                                      ...        Reach               
name                'basin_left1', 246.61  ...    tributary               
chainage                              NaN  ...  175.00000   225.00000     
tag                                        ...   50.0-500.0  50.0-500.0   
2000-02-18 00:06:00             53.065304  ...     0.000000    0.000000   
2000-02-18 00:16:00             53.065304  ...     0.608204    0.517946   

quantity                                                             \
group                                                                 
name                                                                  
chainage            275.00000   350.00000   425.00000   475.00000     
tag                  50.0-500.0  50.0-500.0  50.0-500.0  50.0-500.0   
2000-02-18 00:06:00    0.000000    0.000000    0.000000    0.000000   
2000-02-18 00:16:00    0.449898    0.446345    0.419857    0.417569   

quantity            Variable:TwoTimeSensorGateLevel  \
group                                        Global   
name                                                  
chainage                                        NaN   
tag                                                   
2000-02-18 00:06:00                      107.540001   
2000-02-18 00:16:00                      107.540001   

quantity            Water level:Sensor:s.h.river53745.34  \
group                                             Global   
name                                                       
chainage                                             NaN   
tag                                                        
2000-02-18 00:06:00                            51.905998   
2000-02-18 00:16:00                            51.905998   

quantity            Gate level:Sensor:SensorGateLevel  \
group                                          Global   
name                 

#### .m1d.query()

In [8]:
# Let's get Discharge for the reaches.
df = df.m1d.query("group=='Reach' and quantity=='Discharge'")
df.head(2)

quantity              Discharge                                          \
group                     Reach                                           
name                basin_left1                                           
chainage               1.0020    17.8890   57.3595   101.4935  144.0745   
tag                   0.0-246.6 0.0-246.6 0.0-246.6 0.0-246.6 0.0-246.6   
2000-02-18 00:06:00         0.0       0.0       0.0       0.0       0.0   
2000-02-18 00:16:00        -0.0      -0.0      -0.0      -0.0      -0.0   

quantity                                                                   \
group                                                                       
name                                                                        
chainage             175.4965  197.5490  228.4110    255.6120    274.6260   
tag                 0.0-246.6 0.0-246.6 0.0-246.6 246.6-413.6 246.6-413.6   
2000-02-18 00:06:00       0.0       0.0       0.0         0.0         0.0   
2000-02-18 00:16:00      -0.0      -0.0      -0.0        -0.0        -0.0   

quantity             ...                                              \
group                ...                                               
name                 ...  tributary                                    
chainage             ...   95.0000    110.0000   125.0000   140.0000   
tag                  ... 50.0-500.0 50.0-500.0 50.0-500.0 50.0-500.0   
2000-02-18 00:06:00  ...   0.000000        0.0   0.000000   0.000000   
2000-02-18 00:16:00  ...   0.013857       -0.0   0.050522   0.053607   

quantity                                                                    \
group                                                                        
name                                                                         
chainage              175.0000   225.0000   275.0000   350.0000   425.0000   
tag                 50.0-500.0 50.0-500.0 50.0-500.0 50.0-500.0 50.0-500.0   
2000-02-18 00:06:00   0.000000   0.000000   0.000000   0.000000   0.000000   
2000-02-18 00:16:00   0.090468   0.107519   0.144026   0.177496   0.191049   

quantity                        
group                           
name                            
chainage              475.0000  
tag                 50.0-500.0  
2000-02-18 00:06:00   0.000000  
2000-02-18 00:16:00   0.197755  

[2 rows x 101 columns]

In [9]:
# Let's look at all the reaches with 'trib' in their name.
df = df.m1d.query("name.str.contains('trib')")
df.head(2)

quantity             Discharge                                              \
group                    Reach                                               
name                 tributary                                               
chainage                 70.0       95.0       110.0      125.0      140.0   
tag                 50.0-500.0 50.0-500.0 50.0-500.0 50.0-500.0 50.0-500.0   
2000-02-18 00:06:00   0.000000   0.000000        0.0   0.000000   0.000000   
2000-02-18 00:16:00   0.020551   0.013857       -0.0   0.050522   0.053607   

quantity                                                                    \
group                                                                        
name                                                                         
chainage                 175.0      225.0      275.0      350.0      425.0   
tag                 50.0-500.0 50.0-500.0 50.0-500.0 50.0-500.0 50.0-500.0   
2000-02-18 00:06:00   0.000000   0.000000   0.000000   0.000000   0.000000   
2000-02-18 00:16:00   0.090468   0.107519   0.144026   0.177496   0.191049   

quantity                        
group                           
name                            
chainage                 475.0  
tag                 50.0-500.0  
2000-02-18 00:06:00   0.000000  
2000-02-18 00:16:00   0.197755

In [10]:
# Let's look at the max discharge for each reach.
df.agg(['max'])

quantity  Discharge                                                         \
group         Reach                                                          
name      tributary                                                          
chainage      70.0       95.0       110.0      125.0      140.0      175.0   
tag      50.0-500.0 50.0-500.0 50.0-500.0 50.0-500.0 50.0-500.0 50.0-500.0   
max        2.027786   1.698108   1.452274   1.198294   0.945816   0.696441   

quantity                                                         
group                                                            
name                                                             
chainage      225.0      275.0      350.0      425.0      475.0  
tag      50.0-500.0 50.0-500.0 50.0-500.0 50.0-500.0 50.0-500.0  
max        0.665936    0.59445   0.589273   0.495095   0.296315

In [11]:
# Use some standard pandas methods to format the table a different way
# Tip: Chaining methods in brackets is a great way to explore the data. Comment out lines from bottom up to see the effect.
(
    df.agg(['max'])
    .T
    .droplevel(['group'])
    .unstack()
    #.pipe(lambda df: df * 2) # Uncomment this line to see the effect of the .pipe() method.
)

max
tag                          50.0-500.0
quantity  name      chainage           
Discharge tributary 70.0       2.027786
                    95.0       1.698108
                    110.0      1.452274
                    125.0      1.198294
                    140.0      0.945816
                    175.0      0.696441
                    225.0      0.665936
                    275.0      0.594450
                    350.0      0.589273
                    425.0      0.495095
                    475.0      0.296315

In [12]:
# Let's start from scratch and use bracket chaining to create the same table
df = (
    res.read(column_mode='all')
    .m1d.query("name.str.contains('trib')")
    .m1d.compact()
    .m1d.query("quantity=='Discharge'")
    .agg(['max'])
    .T
    .droplevel('group')
    .unstack()
)
df

max
tag                          50.0-500.0
quantity  name      chainage           
Discharge tributary 70.0       2.027786
                    95.0       1.698108
                    110.0      1.452274
                    125.0      1.198294
                    140.0      0.945816
                    175.0      0.696441
                    225.0      0.665936
                    275.0      0.594450
                    350.0      0.589273
                    425.0      0.495095
                    475.0      0.296315

#### .m1d.agg_chainage()

In [13]:
# Now let's try something different. We aggregate the max discharge for each reach, then look at descriptive staistics along the time axis.
# Here, 'count' is the number of time steps and 'mean' is the mean of the max discharges of all Q-points along a reach.
(
    res.read(column_mode='all')
    .m1d.query("quantity=='Discharge'")
    .m1d.compact()
    .m1d.agg_chainage('max')
    .describe()
)

quantity   Discharge                                                  \
group          Reach                                                   
name     basin_left1                         basin_left2               
tag        0.0-246.6 246.6-413.6 413.6-483.7   0.0-159.1 159.1-441.7   
count      73.000000   73.000000   73.000000        73.0        73.0   
mean        2.649067   -0.360366    0.025040         0.0         0.0   
std         2.888455    0.820759    0.059312         0.0         0.0   
min        -0.000545   -2.174818   -0.009495        -0.0        -0.0   
25%         0.000000   -1.117922   -0.003883         0.0         0.0   
50%         1.183094    0.000000   -0.000986         0.0         0.0   
75%         5.034901    0.000000    0.000000         0.0         0.0   
max         7.773971    1.041439    0.202153        -0.0        -0.0   

quantity                                                             \
group                                                                 
name     basin_right             link_basin_left link_basin_left1_2   
tag      -10.0-636.4 636.4-720.0        0.0-64.0           0.0-58.6   
count      73.000000   73.000000       73.000000               73.0   
mean        4.054045    0.008020        4.492208                0.0   
std         2.918259    0.044539        5.646258                0.0   
min        -0.000000   -0.000000       -2.671227               -0.0   
25%         0.000000    0.000000        0.000000                0.0   
50%         4.485793    0.000000        1.286976                0.0   
75%         5.796013    0.000000        9.329815                0.0   
max         9.116076    0.366389       13.542682               -0.0   

quantity                                                                      \
group                                                                          
name     link_basin_left2_2 link_basin_right           river                   
tag                0.0-69.0         0.0-80.4 53100.0-53950.0 53950.0-54000.0   
count                  73.0        73.000000       73.000000       73.000000   
mean                    0.0         5.737202       72.182541       70.029579   
std                     0.0         3.961021       29.658510       27.060493   
min                    -0.0         0.000000        0.000000        0.000000   
25%                     0.0         0.036112       75.885757       76.290787   
50%                     0.0         6.579083       78.726158       81.531731   
75%                     0.0         8.944139       86.603371       84.478760   
max                    -0.0        11.380744      102.722717       90.085312   

quantity                                                                  \
group                                                                      
name                                                                       
tag      54000.0-54050.0 54050.0-54150.0 54150.0-54350.0 54350.0-55124.2   
count          73.000000       73.000000       73.000000       73.000000   
mean           74.721962       74.971359       75.507965       76.910072   
std            29.563745       29.512341       28.967613       27.771280   
min             0.000000        0.000000       -0.049854        0.000000   
25%            76.206879       75.494148       75.799934       76.689796   
50%            85.145889       84.436142       84.838776       86.002777   
75%            93.835732       94.904182       93.394226       93.584366   
max            99.735207      100.169701      100.301697      100.134666   

quantity             
group                
name      tributary  
tag      50.0-500.0  
count     73.000000  
mean       0.548908  
std        0.489293  
min        0.000000  
25%        0.200483  
50%        0.393951  
75%        0.627040  
max        2.027786

#### .m1d.groupby_chainage()

In [14]:
# Similarly, let's look at the mean of the first and last Q points.
(
    res.read(column_mode='all')
    .m1d.query("quantity=='Discharge'")
    .m1d.compact()
    .m1d.groupby_chainage().nth([0,-1])         # First we groupby, then select the first and last chaianges.
    .describe()
    .droplevel(['quantity','group'], axis=1)
)

name      basin_left1                                                      \
chainage 1.002000     228.411000   255.612000   377.384500   447.534000     
tag         0.0-246.6    0.0-246.6  246.6-413.6  246.6-413.6  413.6-483.7   
count       73.000000    73.000000    73.000000    73.000000    73.000000   
mean        -0.000061     2.306082    -1.521666    -0.416949    -0.049141   
std          0.000171     3.264935     1.941555     0.753996     0.129560   
min         -0.000545    -2.167155    -4.498637    -2.174818    -0.299972   
25%          0.000000     0.000000    -3.195148    -1.117922    -0.122781   
50%          0.000000     1.138214    -0.351519    -0.255342    -0.031151   
75%          0.000000     5.034901     0.000000     0.000000     0.000000   
max          0.000301     7.773971     0.993395     0.848020     0.202153   

name                   basin_left2                                         \
chainage 482.596926   14.597000    126.599500   186.746500   437.951448     
tag       413.6-483.7    0.0-159.1    0.0-159.1  159.1-441.7  159.1-441.7   
count       73.000000         73.0         73.0         73.0         73.0   
mean        -0.001555          0.0          0.0          0.0          0.0   
std          0.004101          0.0          0.0          0.0          0.0   
min         -0.009495         -0.0         -0.0         -0.0         -0.0   
25%         -0.003883          0.0          0.0          0.0          0.0   
50%         -0.000986          0.0          0.0          0.0          0.0   
75%          0.000000          0.0          0.0          0.0          0.0   
max          0.006400         -0.0         -0.0         -0.0         -0.0   

name      ...           river                                                  \
chainage  ...    53975.000000    54025.000000    54075.000000    54125.000000   
tag       ... 53950.0-54000.0 54000.0-54050.0 54050.0-54150.0 54050.0-54150.0   
count     ...       73.000000       73.000000       73.000000       73.000000   
mean      ...       70.029579       74.721962       74.744926       74.757668   
std       ...       27.060493       29.563745       29.496099       29.449482   
min       ...        0.000000        0.000000        0.000000        0.000000   
25%       ...       76.290787       76.206879       75.494148       74.803177   
50%       ...       81.531731       85.145889       84.436142       83.805542   
75%       ...       84.478760       93.835732       94.714996       94.904182   
max       ...       90.085312       99.735207       99.707542      100.169701   

name                                                                      \
chainage    54175.000000    54324.998000    54375.000000    55112.113830   
tag      54150.0-54350.0 54150.0-54350.0 54350.0-55124.2 54350.0-55124.2   
count          73.000000       73.000000       73.000000       73.000000   
mean           75.087944       75.408974       75.508713       76.717072   
std            29.105959       28.908295       28.846025       27.701380   
min            -0.120941       -0.049854       -0.032419        0.000000   
25%            75.529945       75.799934       75.881004       76.689796   
50%            84.253838       84.838776       85.320358       85.784035   
75%            93.394226       93.146133       93.251762       93.584366   
max           100.264687      100.144592      100.042068       99.999596   

name        tributary               
chainage 70.000000    475.000000    
tag        50.0-500.0   50.0-500.0  
count       73.000000    73.000000  
mean        -0.211621     0.188293  
std          1.107742     0.049437  
min         -2.884536     0.000000  
25%         -0.840475     0.164611  
50%         -0.223872     0.186661  
75%          0.340223     0.211712  
max          2.027786     0.296315  

[8 rows x 32 columns]

#### .m1d.groupby()

In [15]:
# We can similarly use the .m1d accessor to group by dimensions other than chainage.
# Below we describe how the global maximum of each quantity in the system varies with time.
(
    res.read(column_mode='all')
    .m1d.groupby('quantity').agg('max')
    .describe()
)

quantity,ControlStrategyId,Discharge,Discharge:Sensor:SensorGauge1,DischargeInStructure,FlowAreaInStructure,FlowVelocity,FlowVelocityInStructure,Gate level:Sensor:SensorGateLevel,GateLevel,ManningResistanceNumber,Variable:TwoTimeSensorGateLevel,Water level:Sensor:s.h.river53745.34,WaterLevel
count,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000
mean,5.917808,79.343079,61.396931,77.747810,47.783684,3.353872,11.329100,53.769997,53.769997,32.665512,107.539993,56.858250,59.200008
std,0.702247,28.940966,26.605530,29.185286,18.166550,0.818552,5.346817,0.000000,0.000000,2.485120,0.000000,1.846656,0.000000
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,53.770000,53.770000,25.000000,107.540001,51.905998,59.200001
25%,6.000000,80.783112,69.281067,76.470184,53.639187,3.652213,11.170768,53.770000,53.770000,33.597073,107.540001,57.567822,59.200001
50%,6.000000,86.655334,74.483246,86.155029,56.015728,3.719032,11.416133,53.770000,53.770000,33.614704,107.540001,57.708385,59.200001
75%,6.000000,99.554710,75.963501,99.477150,57.551910,3.736953,11.514374,53.770000,53.770000,33.618801,107.540001,57.743835,59.200001
max,6.000000,102.722717,77.521797,100.247269,59.036644,3.754651,19.363970,53.770000,53.770000,33.622650,107.540001,57.777584,59.200001
